<a href="https://colab.research.google.com/github/jessicamotam/AvantiBootcamp_ML/blob/main/ML_Classificacao_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: importar dataset do Kaggle

In [32]:
#baixando datasets do kaggle e instalando o pandas
!pip install opendatasets
!pip install pandas

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

In [33]:
import opendatasets as od
import pandas

#importando dataset do COVID
od.download("https://www.kaggle.com/datasets/maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset/data")

Skipping, found downloaded files in "./covid19-ct-scan-lesion-segmentation-dataset" (use force=True to force download)


In [34]:
# listando arquivos e pastas que tem no dataset
!ls

covid19-ct-scan-lesion-segmentation-dataset  sample_data


## Etapa 2: criando dataframe com os metadados da imagem

In [35]:
#importando biblioteca do OPEN CV(biblioteca para processamento de imagem e visão computacional)
  ## cv2 vai permitir tarefas de leitura e gravação de imagens, manipulação de imagens(redimensioamento, transformação e rotação), detecção de objetos, reconhecimento facial
import cv2

#biblioteca para acessar e manipular o sistema de arquivos
import os

In [46]:
# caminho para do diretório no sistema da base do covid / acessando a PASTA FRAMES

root_dir = './covid19-ct-scan-lesion-segmentation-dataset'

In [37]:
#intalando a biblioteca ImageHash -> usada para calculas hashes de imagens

!pip install ImageHash

In [38]:
#bibliotecas

import pandas as pd #manipulação de dados
import imagehash #calcula hashes de imagens(verifica similaridades ou detecta imagens corrompidas)
import matplotlib.pyplot as plt #visualização de graficos
%matplotlib inline
# exibe e gera os gráficos dentro do colab, na célula de baixo

from PIL import Image # importa a biblioteca PIL para manipulação de imagens
contar_imagens_corrompidas = 0
imagens_corrompidas = list() #armazena as imagens corrompidas

In [58]:
dataframe_list = list()

for folder in os.listdir(root_dir):  # Acessando as pastas do diretório
    child_dir = os.path.join(root_dir, folder)

    for image in os.listdir(child_dir):  # Acessando as imagens das pastas
        img_dir = os.path.join(child_dir, image)
        # print(f"Lendo a imagem: {img_dir}")  # Verificando se está lendo todas as imagens das duas pastas

        img = cv2.imread(img_dir)

        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pill = Image.fromarray(img_rgb)

            # Gerando hash da imagem
            img_hash = imagehash.phash(img_pill)
            width, height, channels = img.shape
            img_corrupted = False

            # Adiciona informações da imagem ao dataframe_list
            dataframe_list.append([img_dir, img_corrupted, img_hash, image.split('.')[-1], width, height, channels, folder])
        else:
            contar_imagens_corrompidas += 1
            imagens_corrompidas.append(img_dir)
            img_corrupted = True
            img_hash, image_format, width, height, channels = None, None, None, None, None



## Mostrando se tem imagens corrompidas no DF

In [59]:
print(f'QUANTIDADE DE IMAGENS CORROMPIDAS: {contar_imagens_corrompidas}')
print(f'IMAGENS CORROMPIDAS: {imagens_corrompidas}')
print(f'Total de imagens válidas adicionadas: {len(dataframe_list)}')

QUANTIDADE DE IMAGENS CORROMPIDAS: 0
IMAGENS CORROMPIDAS: []
Total de imagens válidas adicionadas: 5458


In [60]:
df = pd.DataFrame(data=dataframe_list, columns=['image_path', 'imagens_corrompidas', 'image_hash', 'image_format', 'width', 'height', 'channels', 'label'])

df.head()

,image_path,imagens_corrompidas,image_hash,image_format,width,height,channels,label
0,./covid19-ct-scan-lesion-segmentation-dataset/...,False,c63839c63839c7ce,png,512,512,3,masks
1,./covid19-ct-scan-lesion-segmentation-dataset/...,False,926d6d92926d6d92,png,512,512,3,masks
2,./covid19-ct-scan-lesion-segmentation-dataset/...,False,c6c2393fc7c03137,png,512,512,3,masks
3,./covid19-ct-scan-lesion-segmentation-dataset/...,False,c3c31e3c30e1e71e,png,512,512,3,masks
4,./covid19-ct-scan-lesion-segmentation-dataset/...,False,8486737b69a5c6c6,png,512,512,3,masks
